<a href="https://colab.research.google.com/github/brightxzh/movie_review/blob/main/2-1.%EC%96%B8%EC%96%B4%EB%B6%84%EC%84%9D_%ED%99%8D%EC%83%81%EC%88%98_%EA%B0%90%EB%8F%85_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%ED%98%95%ED%83%9C%EC%86%8C_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [ ]:
# konlpy 설치하기
!pip install konlpy

     |████████████████████████████████| 19.4MB 47.3MB/s 
     |████████████████████████████████| 92kB 9.3MB/s 
     |████████████████████████████████| 460kB 48.2MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
# konlpy를 실행하기
import konlpy

In [ ]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [ ]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
## 문서 ID로 실제 파일 불러오기
# 실습을 위한 역대 대통령 취임사: 노무현, 이명박, 박근혜, 문재인
## https://drive.google.com/file/d/1LdK4EJZx_Z-GQPaLLdLM63DD3QpwdLRB/view?usp=sharing

rawdata_downloaded = drive.CreateFile({'id': '1LdK4EJZx_Z-GQPaLLdLM63DD3QpwdLRB'})
rawdata_downloaded.GetContentFile('moviere_홍상수.txt')

In [ ]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('moviere_홍상수.txt', sep="\t")

In [ ]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie,title
0,0,우리동네(nnli****),그때는 맞고 지금은 쳐맞아야지,1,5388,271,2016-06-21 15:03,0,지금은맞고그때는틀리다
1,1,샤막여우(saew****),영화 제목이 의미하는 말 지금(김민희와 불륜)은 맞고 그때(30년전 결혼)는 틀리다...,1,2571,201,2016-06-21 13:11,0,지금은맞고그때는틀리다
2,2,이로사사(dnjs****),바람을피면 그때도맞고 지금도 맞아요,1,1954,138,2016-06-21 14:07,0,지금은맞고그때는틀리다
3,3,바ㄱ수지ㄴ(xoxo****),솔직하셔서 좋으시겠어요.,9,1568,179,2015-09-25 00:07,관람객,지금은맞고그때는틀리다
4,4,kink****,'예술'이라는 같잖은 탈을 쓰고 발정난 추한 늙은이의 비겁한 민낯,1,1348,101,2016-06-21 16:23,0,지금은맞고그때는틀리다
...,...,...,...,...,...,...,...,...,...
4465,1885,신(sino****),김민희배우의인생연기,9,16,38,2017-03-24 00:00,관람객,밤의 해변에서 혼자
4466,1886,toss****,홍상수 감독님 남자로서 존경합니다,10,8,31,2017-03-28 03:39,0,밤의 해변에서 혼자
4467,1887,spmi****,김민희 연기가 많이 늘었다,7,21,44,2017-03-23 18:53,관람객,밤의 해변에서 혼자
4468,1888,마늘(acid****),역시 상수횽이야 고마워,10,25,49,2017-03-23 14:14,관람객,밤의 해변에서 혼자


In [ ]:
원본데이터NAN삭제 = 원본데이터.dropna(axis=0)
## 바로: NAN값(빈값)을 삭제하기 위한 프로세스

In [ ]:
원본데이터NAN삭제

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie,title
0,0,우리동네(nnli****),그때는 맞고 지금은 쳐맞아야지,1,5388,271,2016-06-21 15:03,0,지금은맞고그때는틀리다
1,1,샤막여우(saew****),영화 제목이 의미하는 말 지금(김민희와 불륜)은 맞고 그때(30년전 결혼)는 틀리다...,1,2571,201,2016-06-21 13:11,0,지금은맞고그때는틀리다
2,2,이로사사(dnjs****),바람을피면 그때도맞고 지금도 맞아요,1,1954,138,2016-06-21 14:07,0,지금은맞고그때는틀리다
3,3,바ㄱ수지ㄴ(xoxo****),솔직하셔서 좋으시겠어요.,9,1568,179,2015-09-25 00:07,관람객,지금은맞고그때는틀리다
4,4,kink****,'예술'이라는 같잖은 탈을 쓰고 발정난 추한 늙은이의 비겁한 민낯,1,1348,101,2016-06-21 16:23,0,지금은맞고그때는틀리다
...,...,...,...,...,...,...,...,...,...
4465,1885,신(sino****),김민희배우의인생연기,9,16,38,2017-03-24 00:00,관람객,밤의 해변에서 혼자
4466,1886,toss****,홍상수 감독님 남자로서 존경합니다,10,8,31,2017-03-28 03:39,0,밤의 해변에서 혼자
4467,1887,spmi****,김민희 연기가 많이 늘었다,7,21,44,2017-03-23 18:53,관람객,밤의 해변에서 혼자
4468,1888,마늘(acid****),역시 상수횽이야 고마워,10,25,49,2017-03-23 14:14,관람객,밤의 해변에서 혼자


# 형태소 분석

In [ ]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [ ]:
# "원본데이터NAN삭제"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터NAN삭제.iterrows(): # 원본데이터NAN삭제의 컬럼(행)을 불러와서 반복할 준비!
    대상텍스트 = row[2] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 대상 텍스트의 열은 2번입니다. python은 0부터 숫자를 세게 됩니다!!!
    분류 = row[3] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 평점 정보는 3번째 열에 있습니다.
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [ ]:
# "형태소" 변수의 저장 내용 확인
형태소

[('뭐', 'Noun'),
 ('어떻게', 'Adjective'),
 ('만들었길래', 'Verb'),
 ('평점', 'Noun'),
 ('이', 'Josa'),
 ('7.8', 'Number'),
 ('이나', 'Noun'),
 ('되지', 'Verb'),
 (';;;', 'Punctuation'),
 ('곡성', 'Noun'),
 ('도', 'Josa'),
 ('이', 'Determiner'),
 ('정돈', 'Noun'),
 ('아니었는데', 'Adjective'),
 ('적년', 'Noun'),
 ('까지', 'Josa'),
 ('통틀어', 'Noun'),
 ('서', 'Josa'),
 ('한국영', 'Noun'),
 ('화중', 'Noun'),
 ('제일', 'Noun'),
 ('높은', 'Adjective'),
 ('점수', 'Noun'),
 ('가', 'Josa'),
 ('아닌가', 'Adjective'),
 ('싶다', 'Verb')]

In [ ]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,뭐,Noun,10,1
1,어떻게,Adjective,10,1
2,만들었길래,Verb,10,1
3,평점,Noun,10,1
4,이,Josa,10,1
5,7.8,Number,10,1
6,이나,Noun,10,1
7,되지,Verb,10,1
8,;;;,Punctuation,10,1
9,곡성,Noun,10,1


In [ ]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[      0     1  type  count
 0    그때  Noun     1      1
 1     는  Josa     1      1
 2    맞고  Verb     1      1
 3    지금  Noun     1      1
 4     은  Josa     1      1
 5     쳐  Verb     1      1
 6  맞아야지  Verb     1      1,         0            1  type  count
 0      영화         Noun     1      1
 1      제목         Noun     1      1
 2       이         Josa     1      1
 3      의미         Noun     1      1
 4      하는         Verb     1      1
 5       말         Noun     1      1
 6      지금         Noun     1      1
 7       (  Punctuation     1      1
 8     김민희         Noun     1      1
 9       와         Josa     1      1
 10     불륜         Noun     1      1
 11      )  Punctuation     1      1
 12      은         Noun     1      1
 13     맞고         Verb     1      1
 14     그때         Noun     1      1
 15      (  Punctuation     1      1
 16    30년       Number     1      1
 17      전      Foreign     1      1
 18     결혼         Noun     1      1
 19      )  Punctuation     1      1

In [ ]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,그때,Noun,1,1
1,는,Josa,1,1
2,맞고,Verb,1,1
3,지금,Noun,1,1
4,은,Josa,1,1
...,...,...,...,...
21,높은,Adjective,10,1
22,점수,Noun,10,1
23,가,Josa,10,1
24,아닌가,Adjective,10,1


In [ ]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,그때,Noun,1,1
1,는,Josa,1,1
2,맞고,Verb,1,1
3,지금,Noun,1,1
4,은,Josa,1,1
...,...,...,...,...
21,높은,Adjective,10,1
22,점수,Noun,10,1
23,가,Josa,10,1
24,아닌가,Adjective,10,1


In [ ]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소     품사           분류
!       Punctuation  1     43
                     2      1
                     3      3
                     4      2
                     6      5
                           ..
힘들었던    Adjective    1      1
힘들었습니다  Adjective    5      1
힘들었지만   Adjective    4      1
힘듦에도    Adjective    7      1
힘듭니다    Adjective    9      1
Name: 카운트, Length: 22125, dtype: int64

In [ ]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소    품사          분류     
!      Punctuation 1    43
                   2     1
                   3     3
                   4     2
                   6     5
...                    ...
힘들었던   Adjective   1     1
힘들었습니다 Adjective   5     1
힘들었지만  Adjective   4     1
힘듦에도   Adjective   7     1
힘듭니다   Adjective   9     1

[22125 rows x 1 columns]

In [ ]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')